In [ ]:
from __future__ import print_function, division
import os
os.chdir('siesta_1')
import numpy as np
from sisl import *
import matplotlib.pyplot as plt
%matplotlib inline

# Siesta --- the H2O molecule

This tutorial will describe a complete walk-through of a large fraction of the `sisl` functionalities that may be related to the [Siesta code](http://launchpad.net/siesta).

## Creating the geometry to investigate

Our system of interest will be the $\mathrm H_2\mathrm O$ system. We will start by creating the geometry:

In [ ]:
h2o = Geometry([[0, 0, 0], [0.8, 0.6, 0], [-0.8, 0.6, 0.]], 
               [Atom('O'), Atom('H'), Atom('H')], 
               sc=SuperCell(10, origo=[-5] * 3))

The input are the 1) xyz coordinates, 2) the atomic species and 3) the supercell that is attached.

By printing the object one gets basic information regarding the geometry, such as 1) number of atoms, 2) species of atoms, 3) number of orbitals, 4) orbitals associated with each atom and 5) number of supercells.

In [ ]:
print(h2o)

So there are 3 atoms, 1 Oxygen and 2 Hydrogen. Currently there are only 1 orbital per atom.  
Later we will look into the details of *orbitals* associated with *atoms* and how they may be used for wavefunctions etc.

Lets visualize the atomic positions (here adding atomic indices)

In [ ]:
plt.scatter(h2o.xyz[:, 0], h2o.xyz[:, 1], s=h2o.atom.Z*50);
plt.xlabel(r'$x$'); plt.ylabel(r'$y$');

Now we need to create the input fdf file for Siesta:

In [ ]:
with open('RUN.fdf', 'w') as f:
    f.write("""%include STRUCT.fdf
PAO.BasisSize SZP
SystemLabel siesta_1
MeshCutoff 250. Ry
CDF.Save true
CDF.Compress 9
SaveHS true
SaveRho true
""")
h2o.write('STRUCT.fdf')

The first block of code simply writes a text-file with the required input, the last line of code tells the geometry (`h2o`) to write its information to the file `STRUCT.fdf`. It automatically writes the species blocks etc. to the file.

## Creating the electronic structure

Now run Siesta to obtain the electronic structure.

After having completed the Siesta run we may read the Hamiltonian to manipulate and extract different informations.

In [ ]:
fdf = get_sile('RUN.fdf')
H = fdf.read_hamiltonian()
geom = H.geometry
print(H)

A lot of new information has appeared. First the Hamiltonian object describes the non-orthogonal basis and the "hopping" elements between the orbitals. We see it is a non-orthogonal basis via: `orthogonal: False`. Secondly, we see it was an un-polarized calculation. Lastly the geometry information is printed again. Contrary to the printing of the geometry higher up we now find additional information based on the orbitals on each of the atoms. The oxygen has 9 orbitals while the hydrogens has 4 orbitals. For each orbital one can see its maximal radial part and how charges are initially distributed.

### Plotting orbitals

Often it may be educational (and fun) to plot the orbitals wavefunctions. To do this we use the intrinsic method in the `Orbital` class named `toGrid`. The below code is rather complicated, but that is simply because we want to show the orbitals in a rectangular grid of plots.

In [ ]:
def plot_atom(atom):
    no = len(atom) # number of orbitals
    nx = no // 4
    ny = no // nx
    if nx * ny < no:
        nx += 1
    fig, axs = plt.subplots(nx, ny, figsize=(20, 5*nx))
    fig.suptitle('Atom: {}'.format(atom.symbol), fontsize=14)
    def my_plot(i, orb):
        grid = orb.toGrid(Z=atom.Z)
        # Also write to a cube file
        grid.write('{}_{}.cube'.format(atom.symbol, orb.name()))
        c, r = i // 4, (i - 4) % 4
        if nx == 1:
            ax = axs[r]
        else:
            ax = axs[c][r]
        ax.imshow(grid.grid[:, :, grid.shape[2] // 2])
        ax.set_title(r'${}$'.format(orb.name(True)))
        ax.set_xlabel(r'$x$ [Ang]')
        ax.set_ylabel(r'$y$ [Ang]')
    i = 0
    for orb in atom:
        my_plot(i, orb)
        i += 1
    if i < nx * ny:
        # This removes the empty plots
        for j in range(i, nx * ny):
            c, r = j // 4, (j - 4) % 4
            if nx == 1:
                ax = axs[r]
            else:
                ax = axs[c][r]
            fig.delaxes(ax)
        plt.draw()
plot_atom(geom.atom[0])
plot_atom(geom.atom[1])

## Hamiltonian eigenstates

At this point we have the full Hamiltonian as well as the basis functions used in the Siesta calculation.
This completes what is needed to calculate a great deal of physical quantities, e.g. eigenstates, density of states, projected density of states and wavefunctions.

To begin with we calculate the $\Gamma$-point eigenstates and plot a subset of the eigenstates' norm on the geometry.

In [ ]:
es = H.eigenstate()
h2o = H.geometry
h2o.sc.origo = [-4, -4, -4]
print(h2o.sc, h2o.sc.origo)
O_idx = h2o.a2o(0, True)
H1_idx = h2o.a2o(1, True)
H2_idx = h2o.a2o(2, True)
# Reduce the contained eigenstates to only the HOMO and LUMO
idx_homo = (es.eig > 0).nonzero()[0][0] - 1
es = es.sub(np.arange(idx_homo, idx_homo + 2))
_, ax = plt.subplots(1, 2, figsize=(14, 2));
for i, (n, c) in enumerate(zip(es.norm2(sum=False), 'rbg')):
    ax[i].scatter(h2o.xyz[0, 0], h2o.xyz[0, 1], 600 * n[O_idx].sum(), facecolor=c, alpha=0.5);
    ax[i].scatter(h2o.xyz[1, 0], h2o.xyz[1, 1], 600 * n[H1_idx].sum(), facecolor=c, alpha=0.5);
    ax[i].scatter(h2o.xyz[2, 0], h2o.xyz[2, 1], 600 * n[H2_idx].sum(), facecolor=c, alpha=0.5);

These are not that interesting. The projection of the HOMO and LUMO states show where the largest weight of the HOMO and LUMO states, however we can't see the symmetry differences between the HOMO and LUMO states.

Instead of plotting the weight on each orbital it is more interesting to plot the actual wavefunctions which contains the orbital symmetries, however, matplotlib is currently not capable of plotting real-space iso-surface plots. To do this, please use VMD or your preferred software.

In [ ]:
def integrate(g):
    print('Real space integrated wavefunction: {:.4f}'.format((np.absolute(g.grid) ** 2).sum() * g.dvolume))
g = Grid(0.2, sc=h2o.sc)
es.sub(0).wavefunction(g, eta=True)
integrate(g)
#g.write('HOMO.cube')
g.fill(0)
es.sub(1).wavefunction(g, eta=True)
integrate(g)
#g.write('LUMO.cube')

## Real space charge

Since we have the basis functions we can also plot the charge in the grid. We can do this via either reading the density matrix or read in the charge output from Siesta directly.
Since both should yield the same value we can compare the output from Siesta with that calculated in sisl.

You will notice that re-creating the density on a real space grid in sisl is much slower than creating the wavefunction. This is because we need _orbital multiplications_.

In [ ]:
DM = fdf.read_density_matrix()
rho = get_sile('siesta_1.nc').read_grid('Rho')

In [ ]:
# Now calculate the density on the same grid using sisl and compare to Siesta output
DM_rho = rho.copy()
DM_rho.fill(0)
DM.density(DM_rho, eta=True)
diff = DM_rho - rho
print('Real space integrated density difference: {:.3e}'.format(diff.grid.sum() * diff.dvolume))